In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers.core import Dense
from keras.layers.core import Flatten
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.preprocessing import image

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir='/content/drive/MyDrive/summer intern/dataset/train'

In [ ]:
test_data_dir='/content/drive/MyDrive/summer intern/dataset/test'

In [ ]:
img_height,img_width=224,224
img_path=data_dir+'/0/1.png'
img=image.load_img(img_path,target_size=(img_height,img_width))
img_data=image.img_to_array(img)
img_data=np.expand_dims(img_data,axis=0)
img_data=preprocess_input(img_data)

In [ ]:
classes=[str(i) for i in range(6)]

In [ ]:
val_data_dir='/content/drive/MyDrive/summer intern/dataset/valid'

In [ ]:
train_images_per_class=500

In [ ]:
train_data=[]
train_label=[]
img_height,img_width=224,224
for cls in classes:
  print(cls)
  for i in range(500):
    img_path=data_dir+'/'+cls+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    train_data.append(img_data)
    train_label.append(int(cls))


0
1
2
3
4
5


TypeError: ignored

In [ ]:
train_label[:2*train_images_per_class]=[0]*2*train_images_per_class
train_label[2*train_images_per_class:]=[1]*4*train_images_per_class

In [ ]:
valid_images_per_class=100

In [ ]:
valid_data=[]
valid_label=[]
img_height,img_width=224,224
for cls in classes:
  print(cls)
  for i in range(100):
    img_path=val_data_dir+'/'+cls+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    valid_data.append(img_data)
    valid_label.append(int(cls))


0
1
2
3
4
5


In [ ]:
valid_label=[]
valid_label[:2*valid_images_per_class]=[0]*2*valid_images_per_class
valid_label[2*valid_images_per_class:]=[1]*4*valid_images_per_class

In [ ]:
train_data=np.asarray(train_data)
train_label=np.asarray(train_label)

In [ ]:
valid_data=np.asarray(valid_data)
valid_label=np.asarray(valid_label)

In [ ]:
def preprocess_data(X,Y,n):
  X_p=preprocess_input(X)
  Y_p=tf.keras.utils.to_categorical(Y,n)
  return (X_p,Y_p)

In [ ]:
train_data,train_label=preprocess_data(train_data,train_label,2)

In [ ]:
valid_data,valid_label=preprocess_data(valid_data,valid_label,2)

In [ ]:
print(train_data.shape,train_label.shape)

(3000, 224, 224, 3) (3000, 2)


In [ ]:
resnet_model = Sequential()
input_t=tf.keras.Input(shape=(224,224,3))
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_tensor=input_t,
                   pooling='max',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,637,826
Trainable params: 1,050,114
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
resnet_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

history = resnet_model.fit(train_data,train_label, validation_data=(valid_data,valid_label),batch_size=32, epochs=25,verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/25
94/94 [==============================] - 14s 124ms/step - loss: 0.7086 - accuracy: 0.8593 - val_loss: 0.6355 - val_accuracy: 0.8083
Epoch 2/25
94/94 [==============================] - 11s 112ms/step - loss: 0.2671 - accuracy: 0.8913 - val_loss: 0.6931 - val_accuracy: 0.7567
Epoch 3/25
94/94 [==============================] - 11s 113ms/step - loss: 0.2737 - accuracy: 0.8973 - val_loss: 0.5958 - val_accuracy: 0.7717
Epoch 4/25
94/94 [==============================] - 11s 112ms/step - loss: 0.2424 - accuracy: 0.9033 - val_loss: 0.5651 - val_accuracy: 0.7917
Epoch 5/25
94/94 [==============================] - 11s 114ms/step - loss: 0.2026 - accuracy: 0.9113 - val_loss: 0.6926 - val_accuracy: 0.8150
Epoch 6/25
94/94 [==============================] - 11s 114ms/step - loss: 0.2517 - accuracy: 0.9010 - val_loss: 0.5466 - val_accuracy: 0.8000
Epoch 7/25
94/94 [==============================] - 11s 115ms/step - loss: 0.1935 - accuracy: 0.9173 - val_loss: 0.5225 - val_accuracy: 0.8000

KeyboardInterrupt: ignored

In [ ]:
test_images_per_class=100

In [ ]:
test_data=[]
test_label=[]
img_height,img_width=224,224
for cls in classes:
  print(cls)
  for i in range(100):
    img_path=test_data_dir+'/'+cls+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    test_data.append(img_data)
    test_label.append(int(cls))


0
1
2
3
4
5


In [ ]:
test_label=[]
test_label[:2*test_images_per_class]=[0]*2*test_images_per_class
test_label[2*test_images_per_class:]=[1]*4*test_images_per_class

In [ ]:
test_data=np.asarray(test_data)
test_label=np.asarray(test_label)

In [ ]:
test_data,test_label=preprocess_data(test_data,test_label,2)

In [ ]:
test_data.shape,test_label.shape

((600, 224, 224, 3), (600, 2))

In [ ]:
predictions=resnet_model.predict(test_data,verbose=1)

19/19 [==============================] - 2s 92ms/step


In [ ]:
test_loss,test_acc=resnet_model.evaluate(test_data,test_label,verbose=1)

19/19 [==============================] - 2s 95ms/step - loss: 1.1616 - accuracy: 0.8300


In [ ]:
print('Testing data accuracy = '+str(test_acc)+'\n Testing data loss = '+str(test_loss))

Testing data accuracy = 0.8299999833106995
 Testing data loss = 1.161588191986084


In [ ]:
predictions

array([[1.0000000e+00, 1.4599439e-09],
       [1.4173168e-03, 9.9858272e-01],
       [2.6640040e-04, 9.9973363e-01],
       ...,
       [3.5769554e-04, 9.9964225e-01],
       [3.6551523e-06, 9.9999630e-01],
       [1.2154990e-05, 9.9998784e-01]], dtype=float32)

In [ ]:
test_shape=test_data.shape

In [ ]:
test_data=test_data.reshape((-1,1))

In [ ]:
np.savetxt('/content/drive/MyDrive/summer intern/test_label.txt',test_label)

In [ ]:
np.savetxt('/content/drive/MyDrive/summer intern/test_data.txt',test_data)

In [ ]:
os.getcwd()

'/content'

In [ ]:
from tensorflow.keras.models import model_from_json

In [ ]:
model_json = resnet_model.to_json()
with open("/content/drive/MyDrive/summer intern/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
resnet_model.save_weights("/content/drive/MyDrive/summer intern/model.h5")
print("Saved model to disk")
 
# later...
 


Saved model to disk


In [ ]:
# load json and create model
json_file = open('/content/drive/MyDrive/summer intern/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/summer intern/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Loaded model from disk


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Model 1

In [ ]:
train_data_1=[]
img_height,img_width=224,224
for cls in range(2):
  print(cls)
  for i in range(500):
    img_path=data_dir+'/'+str(cls)+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    train_data_1.append(img_data)

0
1


In [ ]:
len(train_data_1)

1000

In [ ]:
train_label_1=[]
train_label_1[:train_images_per_class]=[0]*train_images_per_class
train_label_1[train_images_per_class:]=[1]*train_images_per_class

In [ ]:
train_data_1=np.asarray(train_data_1)
train_label_1=np.asarray(train_label_1)

In [ ]:
valid_data_1=[]
valid_label_1=[]
img_height,img_width=224,224
for cls in range(2):
  print(cls)
  for i in range(100):
    img_path=val_data_dir+'/'+str(cls)+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    valid_data_1.append(img_data)

valid_label_1[:valid_images_per_class]=[0]*valid_images_per_class
valid_label_1[valid_images_per_class:]=[1]*valid_images_per_class

0
1


In [ ]:
valid_data_1=np.asarray(valid_data_1)
valid_label_1=np.asarray(valid_label_1)

In [ ]:
train_data_1,train_label_1=preprocess_data(train_data_1,train_label_1,2)

In [ ]:
valid_data_1,valid_label_1=preprocess_data(valid_data_1,valid_label_1,2)

In [ ]:
valid_label_1.shape

(200, 2)

In [ ]:
print(train_data_1.shape,train_label_1.shape)

(1000, 224, 224, 3) (1000, 2)


In [ ]:
resnet_model_1 = Sequential()
input_t=tf.keras.Input(shape=(224,224,3))
pretrained_model_1= tf.keras.applications.ResNet50(include_top=False,
                   input_tensor=input_t,
                   pooling='max',classes=2,
                   weights='imagenet')
for layer in pretrained_model_1.layers:
        layer.trainable=False

resnet_model_1.add(pretrained_model_1)

In [ ]:
resnet_model_1.add(Flatten())
resnet_model_1.add(Dense(512, activation='relu'))
resnet_model_1.add(Dense(2, activation='softmax'))

In [ ]:
resnet_model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               1049088   
                                                                 
 dense_5 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,637,826
Trainable params: 1,050,114
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
resnet_model_1.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

history_1 = resnet_model_1.fit(train_data_1,train_label_1,validation_data=(valid_data_1,valid_label_1),batch_size=32, epochs=25,verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/25
32/32 [==============================] - 16s 413ms/step - loss: 5.6128 - accuracy: 0.6730 - val_loss: 1.3829 - val_accuracy: 0.7550
Epoch 2/25
32/32 [==============================] - 6s 190ms/step - loss: 0.4940 - accuracy: 0.8400 - val_loss: 0.3370 - val_accuracy: 0.8850
Epoch 3/25
32/32 [==============================] - 8s 256ms/step - loss: 0.3625 - accuracy: 0.8530 - val_loss: 0.3272 - val_accuracy: 0.8500
Epoch 4/25
32/32 [==============================] - 4s 140ms/step - loss: 0.2470 - accuracy: 0.8930 - val_loss: 0.2542 - val_accuracy: 0.8850
Epoch 5/25
32/32 [==============================] - 6s 180ms/step - loss: 0.2074 - accuracy: 0.8920 - val_loss: 0.3715 - val_accuracy: 0.8400
Epoch 6/25
32/32 [==============================] - 7s 236ms/step - loss: 0.2168 - accuracy: 0.9010 - val_loss: 0.3303 - val_accuracy: 0.8500
Epoch 7/25
32/32 [==============================] - 6s 200ms/step - loss: 0.2366 - accuracy: 0.8930 - val_loss: 0.2780 - val_accuracy: 0.8800
Epoch

KeyboardInterrupt: ignored

In [ ]:
test_data_1=[]
img_height,img_width=224,224
for cls in range(2):
  print(cls)
  for i in range(100):
    img_path=test_data_dir+'/'+str(cls)+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    test_data_1.append(img_data)

0
1


In [ ]:
test_label_1=[]
test_label_1[:test_images_per_class]=[0]*test_images_per_class
test_label_1[test_images_per_class:]=[1]*test_images_per_class

In [ ]:
test_data_1=np.asarray(test_data_1)
test_label_1=np.asarray(test_label_1)

In [ ]:
test_data_1,test_label_1=preprocess_data(test_data_1,test_label_1,2)

In [ ]:
test_data_1.shape,test_label_1.shape

((200, 224, 224, 3), (200, 2))

In [ ]:
predictions_1=resnet_model_1.predict(test_data_1,verbose=1)

7/7 [==============================] - 22s 3s/step


In [ ]:
test_loss_1,test_acc_1=resnet_model_1.evaluate(test_data_1,test_label_1,verbose=1)

7/7 [==============================] - 2s 293ms/step - loss: 0.1933 - accuracy: 0.9050


In [ ]:
print('Testing data accuracy = '+str(test_acc_1)+'\n Testing data loss = '+str(test_loss_1))

Testing data accuracy = 0.9049999713897705
 Testing data loss = 0.19332155585289001


In [ ]:
model_json_1 = resnet_model_1.to_json()
with open("/content/drive/MyDrive/summer intern/model_1.json", "w") as json_file:
    json_file.write(model_json_1)
# serialize weights to HDF5
resnet_model_1.save_weights("/content/drive/MyDrive/summer intern/model_1.h5")
print("Saved model to disk")

Saved model to disk


## Model 2

In [ ]:
train_data_2=[]
img_height,img_width=224,224
for cls in range(2,6):
  print(cls)
  for i in range(500):
    img_path=data_dir+'/'+str(cls)+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    train_data_2.append(img_data)

2
3
4
5


In [ ]:
len(train_data_2)

2000

In [ ]:
train_label_2=[]
train_label_2[:train_images_per_class]=[0]*train_images_per_class
train_label_2[train_images_per_class:2*train_images_per_class]=[1]*train_images_per_class
train_label_2[2*train_images_per_class:3*train_images_per_class]=[2]*train_images_per_class
train_label_2[3*train_images_per_class:4*train_images_per_class]=[3]*train_images_per_class

In [ ]:
train_data_2=np.asarray(train_data_2)
train_label_2=np.asarray(train_label_2)

In [ ]:
valid_data_2=[]
img_height,img_width=224,224
for cls in range(2,6):
  print(cls)
  for i in range(100):
    img_path=val_data_dir+'/'+str(cls)+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    valid_data_2.append(img_data)


2
3
4
5


In [ ]:
valid_label_2=[]
valid_label_2[:valid_images_per_class]=[0]*valid_images_per_class
valid_label_2[valid_images_per_class:2*valid_images_per_class]=[1]*valid_images_per_class
valid_label_2[2*valid_images_per_class:3*valid_images_per_class]=[2]*valid_images_per_class
valid_label_2[3*valid_images_per_class:4*valid_images_per_class]=[3]*valid_images_per_class

In [ ]:
valid_data_2=np.asarray(valid_data_2)
valid_label_2=np.asarray(valid_label_2)

In [ ]:
print(train_data_2.shape,train_label_2.shape)

(2000, 224, 224, 3) (2000,)


In [ ]:
print(valid_data_2.shape,valid_label_2.shape)

(400, 224, 224, 3) (400,)


In [ ]:
train_data_2,train_label_2=preprocess_data(train_data_2,train_label_2,4)

In [ ]:
valid_data_2,valid_label_2=preprocess_data(valid_data_2,valid_label_2,4)

In [ ]:
print(valid_data_2.shape,valid_label_2.shape)

(400, 224, 224, 3) (400, 4)


In [ ]:
print(train_data_2.shape,train_label_2.shape)

(2000, 224, 224, 3) (2000, 4)


In [ ]:
resnet_model_2 = Sequential()
input_t=tf.keras.Input(shape=(224,224,3))
pretrained_model_2= tf.keras.applications.ResNet50(include_top=False,
                   input_tensor=input_t,
                   pooling='max',classes=4,
                   weights='imagenet')
for layer in pretrained_model_2.layers:
        layer.trainable=False

resnet_model_2.add(pretrained_model_2)

In [ ]:
resnet_model_2.add(Flatten())
resnet_model_2.add(Dense(512, activation='relu'))
resnet_model_2.add(Dense(4, activation='softmax'))

In [ ]:
resnet_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dense_3 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 24,638,852
Trainable params: 1,051,140
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
resnet_model_2.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

history_2 = resnet_model_2.fit(train_data_2,train_label_2,validation_data=(valid_data_2,valid_label_2),batch_size=32, epochs=25,verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/25
63/63 [==============================] - 51s 139ms/step - loss: 3.9420 - accuracy: 0.5005 - val_loss: 0.9948 - val_accuracy: 0.6125
Epoch 2/25
63/63 [==============================] - 8s 121ms/step - loss: 0.9534 - accuracy: 0.6150 - val_loss: 0.9530 - val_accuracy: 0.6100
Epoch 3/25
63/63 [==============================] - 8s 122ms/step - loss: 0.9407 - accuracy: 0.6585 - val_loss: 1.3662 - val_accuracy: 0.5775
Epoch 4/25
63/63 [==============================] - 8s 122ms/step - loss: 0.8000 - accuracy: 0.6785 - val_loss: 1.2331 - val_accuracy: 0.5875
Epoch 5/25
63/63 [==============================] - 8s 120ms/step - loss: 0.6744 - accuracy: 0.7290 - val_loss: 0.8929 - val_accuracy: 0.6475
Epoch 6/25
63/63 [==============================] - 7s 119ms/step - loss: 0.7467 - accuracy: 0.7095 - val_loss: 0.9942 - val_accuracy: 0.6350
Epoch 7/25
63/63 [==============================] - 7s 119ms/step - loss: 0.7413 - accuracy: 0.7195 - val_loss: 0.8953 - val_accuracy: 0.6450
Epoch

KeyboardInterrupt: ignored

In [ ]:
test_data_2=[]
img_height,img_width=224,224
for cls in range(2,6):
  print(cls)
  for i in range(100):
    img_path=test_data_dir+'/'+str(cls)+'/'+str(i)+'.png'
    img=image.load_img(img_path,target_size=(img_height,img_width))
    img_data=image.img_to_array(img)
    test_data_2.append(img_data)

2
3
4
5


In [ ]:
test_label_2=[]
test_label_2[:test_images_per_class]=[0]*test_images_per_class
test_label_2[test_images_per_class:2*test_images_per_class]=[1]*test_images_per_class
test_label_2[2*test_images_per_class:3*test_images_per_class]=[2]*test_images_per_class
test_label_2[3*test_images_per_class:4*test_images_per_class]=[3]*test_images_per_class

In [ ]:
test_data_2=np.asarray(test_data_2)
test_label_2=np.asarray(test_label_2)

In [ ]:
test_data_2,test_label_2=preprocess_data(test_data_2,test_label_2,4)

In [ ]:
test_data_2.shape,test_label_2.shape

((400, 224, 224, 3), (400, 4))

In [ ]:
predictions_2=resnet_model_2.predict(test_data_2,verbose=1)

13/13 [==============================] - 2s 95ms/step


In [ ]:
test_loss_2,test_acc_2=resnet_model_2.evaluate(test_data_2,test_label_2,verbose=1)

13/13 [==============================] - 1s 97ms/step - loss: 0.9776 - accuracy: 0.6350


In [ ]:
print('Testing data accuracy = '+str(test_acc_2)+'\n Testing data loss = '+str(test_loss_2))

Testing data accuracy = 0.6349999904632568
 Testing data loss = 0.977587878704071


In [ ]:
model_json_2 = resnet_model_2.to_json()
with open("/content/drive/MyDrive/summer intern/model_2.json", "w") as json_file:
    json_file.write(model_json_2)
# serialize weights to HDF5
resnet_model_2.save_weights("/content/drive/MyDrive/summer intern/model_2.h5")
print("Saved model to disk")

Saved model to disk
